### We've labelled 144 images (train), 18 images (valid), and 12 images (test). After filtering the post event images with building ROIs (10730 -> 4257), how many images left for us to label?

- answer: 4163

In [29]:
import os
import shutil
from pathlib import Path

import pandas as pd

In [2]:
ROOT_PATH = Path(os.getcwd()).parent.as_posix()
DATASET_PATH = Path(ROOT_PATH, "datasets/storm-damage-detection/working").as_posix()
BUILDING_COVERAGE_PATH = Path(DATASET_PATH, "unprep/building_footprint_roi/building_coverage.csv").as_posix()

In [22]:
finished_valid = Path(DATASET_PATH, "sdd/valid/images")
finished_train = Path(DATASET_PATH, "sdd/train/images")
finished = os.listdir(finished_valid) + os.listdir(finished_train)
finished = set([int(x.split(".")[0].split("_")[2]) for x in finished])
print(len(finished))

162


In [23]:
finished

{0,
 1,
 2,
 3,
 4,
 7,
 9,
 11,
 13,
 16,
 18,
 19,
 20,
 21,
 23,
 29,
 30,
 32,
 34,
 35,
 36,
 37,
 38,
 40,
 42,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 64,
 66,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 76,
 77,
 79,
 81,
 82,
 84,
 88,
 89,
 92,
 97,
 5092,
 5356,
 5399,
 5485,
 5550,
 5668,
 5722,
 5732,
 5882,
 5974,
 6180,
 6213,
 6261,
 6398,
 6637,
 6808,
 6878,
 7091,
 7122,
 7341,
 7371,
 7395,
 7518,
 7594,
 7790,
 7839,
 7892,
 7931,
 8049,
 8058,
 8072,
 8081,
 8097,
 8133,
 8146,
 8180,
 8239,
 8249,
 8285,
 8288,
 8291,
 8306,
 8311,
 8314,
 8345,
 8350,
 8375,
 8384,
 8408,
 8431,
 8451,
 8462,
 8464,
 8481,
 8508,
 8531,
 8536,
 8549,
 8575,
 8594,
 8596,
 8613,
 8620,
 8621,
 8631,
 8675,
 8688,
 8691,
 8702,
 8707,
 8744,
 8768,
 8800,
 8809,
 8828,
 8848,
 8855,
 8872,
 8878,
 8883,
 8890,
 8957,
 8959,
 8972,
 8974,
 8988,
 9001,
 9033,
 9082,
 9087,
 9183,
 9249,
 9275,
 9425,
 9442,
 9479,
 9659,
 9734,
 9788,
 9795,
 9821,


In [12]:
filtered = pd.read_csv(BUILDING_COVERAGE_PATH)
filtered = filtered.loc[filtered["coverage"] >= 0.1]
filtered

,file_index,has_building,coverage,dummy_fname
1,1,True,0.385014,tile_19_62.jpg
2,2,True,0.302933,tile_36_81.jpg
5,5,True,0.139576,tile_42_66.jpg
7,7,True,0.183407,tile_31_88.jpg
9,9,True,0.258347,tile_10_46.jpg
...,...,...,...,...
10720,9990,True,0.139694,tile_41_100.jpg
10726,9996,True,0.398483,tile_46_30.jpg
10727,9997,True,0.146816,tile_14_38.jpg
10728,9998,True,0.387959,tile_5_75.jpg


In [35]:
# leftover images to annotate
# - not in finished (not labelled with roboflow yet)
# - in filtered (it's an image with >= 0.1 building coverage)
leftover = filtered.loc[~filtered["file_index"].isin(finished)]
leftover_ixs = set(leftover["file_index"].tolist())
leftover

,file_index,has_building,coverage,dummy_fname
5,5,True,0.139576,tile_42_66.jpg
85,85,True,0.330040,tile_14_36.jpg
102,10000,True,0.379562,tile_21_83.jpg
108,10006,True,0.382275,tile_13_31.jpg
109,10007,True,0.327053,tile_55_64.jpg
...,...,...,...,...
10720,9990,True,0.139694,tile_41_100.jpg
10726,9996,True,0.398483,tile_46_30.jpg
10727,9997,True,0.146816,tile_14_38.jpg
10728,9998,True,0.387959,tile_5_75.jpg


In [39]:
ORI_POST_EVENT_PATH = Path(DATASET_PATH, "unprep/all_grids/post_event")
NEW_POST_EVENT_PATH = Path(DATASET_PATH, "unprep/all_grids/post_event_filtered")
shutil.rmtree(NEW_POST_EVENT_PATH, ignore_errors=True)
os.makedirs(NEW_POST_EVENT_PATH, exist_ok=True)

In [40]:
count = 0
for fname in os.listdir(ORI_POST_EVENT_PATH):
    file_ix = int(fname.split("_")[-1].rstrip(".jpg"))
    
    if not file_ix in leftover_ixs:
        continue

    # copy file to the new path
    shutil.copyfile(Path(ORI_POST_EVENT_PATH, fname), Path(NEW_POST_EVENT_PATH, fname))
    count += 1
print(f"Copied {count} files!")

Copied 4163 files!
